Засоби підготовки та аналізу даних

Лабораторна робота №2 Наука про дані: підготовчий етап

Мета роботи: ознайомитися з основними кроками по роботі з даними – workflow
від постановки задачі до написання пояснювальної записки, зрозуміти постановку задачі
та природу даних, над якими виконується аналітичні операції
Основні поняття: сирі дані (raw data), підготовка даних (data preparation)

Імпорт бібліотек

In [24]:
import os
import datetime
import urllib.request
import requests
import pandas as pd

Індекс (номер) області + дата та час


In [25]:
def download_vhi(province_id, save_dir="data"):
    os.makedirs(save_dir, exist_ok=True)
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    url = (
        f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?"
        f"country=UKR&provinceID={province_id}&year1=1981&year2=2024&type=Mean"
    )
    filename = f"vhi_id_{province_id}_{timestamp}.csv"
    filepath = os.path.join(save_dir, filename)

    if not os.path.exists(filepath):
        with urllib.request.urlopen(url) as response, open(filepath, 'wb') as out_file:
            out_file.write(response.read())
        print(f"Завантажено файл: {filename}")
    else:
        print(f"Файл уже існує: {filename}")

for province_id in range(1, 26):
    download_vhi(province_id)

Завантажено файл: vhi_id_1_20250603_034048.csv
Завантажено файл: vhi_id_2_20250603_034049.csv
Завантажено файл: vhi_id_3_20250603_034050.csv
Завантажено файл: vhi_id_4_20250603_034051.csv
Завантажено файл: vhi_id_5_20250603_034053.csv
Завантажено файл: vhi_id_6_20250603_034054.csv
Завантажено файл: vhi_id_7_20250603_034055.csv
Завантажено файл: vhi_id_8_20250603_034056.csv
Завантажено файл: vhi_id_9_20250603_034058.csv
Завантажено файл: vhi_id_10_20250603_034059.csv
Завантажено файл: vhi_id_11_20250603_034100.csv
Завантажено файл: vhi_id_12_20250603_034102.csv
Завантажено файл: vhi_id_13_20250603_034103.csv
Завантажено файл: vhi_id_14_20250603_034104.csv
Завантажено файл: vhi_id_15_20250603_034105.csv
Завантажено файл: vhi_id_16_20250603_034107.csv
Завантажено файл: vhi_id_17_20250603_034108.csv
Завантажено файл: vhi_id_18_20250603_034109.csv
Завантажено файл: vhi_id_19_20250603_034110.csv
Завантажено файл: vhi_id_20_20250603_034112.csv
Завантажено файл: vhi_id_21_20250603_034113.csv
З

Зчитуємо файли

In [26]:
def read_vhi_files(folder="data"):
    files = [f for f in os.listdir(folder) if f.endswith(".csv") and not f.startswith("df_all")]
    dfs = []

    for file in files:
        try:
            df = pd.read_csv(os.path.join(folder, file), header=1, skiprows=1)

            if df.shape[1] == 9:
                df.columns = ["index", "year", "week", "SMN", "SMT", "VCI", "TCI", "VHI", "drop"]
                df = df.drop(columns=["index", "drop"], errors="ignore")
            elif df.shape[1] == 8:
                df.columns = ["year", "week", "SMN", "SMT", "VCI", "TCI", "VHI", "drop"]
                df = df.drop(columns=["drop"], errors="ignore")
            else:
                print(f"Пропущено файл з некоректною кількістю колонок: {file}")
                continue

            df["VHI"] = pd.to_numeric(df["VHI"], errors="coerce")
            df = df[df["VHI"] != -1].dropna()

            if "vhi_id_" in file:
                df["area"] = int(file.split("_")[2])
            else:
                continue

            dfs.append(df)
        except Exception as e:
            print(f"Помилка при обробці {file}: {e}")

    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

df_all = read_vhi_files("data")

Зміна індексів

In [27]:
df_all = df_all.rename(columns={"year": "Year"})
df_all["Year"] = pd.to_numeric(df_all["Year"], errors="coerce").astype("Int64")

area_map = {
    1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21,
    11: 9, 13: 10, 14: 11, 15: 12, 16: 13, 17: 15, 18: 14, 19: 16, 21: 17,
    22: 18, 23: 6, 24: 1, 25: 2
}


df_all.to_csv("data/df_all.csv", index=False)
print("Дані збережено як data/df_all.csv")

Дані збережено як data/df_all.csv


Ряд VHI для області за вказаний рік, пошук екстремумів (min та max):

In [28]:
def vhi(area, year):
    return df_all[(df_all["area"] == area) & (df_all["Year"] == year)]["VHI"]

def vhi_min(area, year):
    return df_all[(df_all["area"] == area) & (df_all["Year"] == year)]["VHI"].min()

def vhi_max(area, year):
    return df_all[(df_all["area"] == area) & (df_all["Year"] == year)]["VHI"].max()

def vhi_range(year_min, year_max, areas):
    if not isinstance(areas, list) or not areas:
        return print("Empty or not a list")
    return df_all[
        (df_all["Year"] >= year_min) &
        (df_all["Year"] <= year_max) &
        (df_all["area"].isin(areas))
    ][["Year", "VHI", "area"]]

df_all['Year'] = df_all['Year'].astype(int)

In [29]:
vhi(6, 2005)

92916    69.37
92917    74.87
92918    76.84
92919    75.70
92920    75.60
         ...  
95147    33.44
95148    34.77
95149    33.20
95150    35.88
95151    40.18
Name: VHI, Length: 102, dtype: float64

In [30]:
vhi_min(8, 2005)

38.47

In [31]:
vhi_max(8, 2005)

77.32

In [32]:
vhi_range(2000, 2005, [9, 2, 3])

,Year,VHI,area
75195,2000,24.84,2
75196,2000,24.58,2
75197,2000,26.47,2
75198,2000,29.62,2
75199,2000,32.73,2
...,...,...,...
108257,2005,31.74,9
108258,2005,33.62,9
108259,2005,35.55,9
108260,2005,36.42,9


Екстремальні посухи:

In [33]:
def extreme_droughts(percent):
    df_d = df_all[(df_all["VHI"] <= 15) & (df_all["VHI"] != -1)]
    grouped = df_d.groupby("Year")["area"].nunique()
    return grouped[grouped > (25 * percent / 100)].reset_index()

In [34]:
extreme_droughts(5)

,Year,area
0,1986,2
1,1993,2
2,2000,6
3,2007,4


Помірні посухи:

In [35]:
def moderate_droughts(percent, vmin=15, vmax=40):
    df_d = df_all[(df_all["VHI"] >= vmin) & (df_all["VHI"] <= vmax)]
    grouped = df_d.groupby("Year")["area"].nunique()
    total = df_all["area"].nunique()
    return grouped[grouped > (total * percent / 100)].reset_index()

In [36]:
moderate_droughts(25)

,Year,area
0,1982,25
1,1983,24
2,1984,25
3,1985,25
4,1986,25
5,1987,25
6,1988,25
7,1989,25
8,1990,25
9,1991,24
